# Download, parse, extract Ontario human IAV from NCBI Influenza DB

- Download all nucleotide sequences in the NCBI Influenza DB 
- Parse Human HA gene sequences from Ontario beloning to H1 or H3
- Group sequences by subtype and flu season
- Perform multiple sequence alignments (MSA)


In [70]:
import re
from pathlib import Path

import pandas as pd
import numpy as np

from Bio import SeqIO
from Bio import Entrez

Email is required to use NCBI Entrez API

In [13]:
Entrez.email = 'peter.kruczkiewicz@canada.ca'

Downloaded all Influenza nucleotide sequences from the NCBI FTP site for the NCBI Influenza DB (2019-09-26T08:51:00+5):

```bash
wget ftp://ftp.ncbi.nih.gov/genomes/INFLUENZA/influenza.fna.gz
```

Output FASTA headers for H1 or H3 HA genes (`(segment 4|\(HA\)|hemagglutinin)`) Influenza A from Ontario (`(Ontario|Toronto|Canada-ON)`) to `ontario-H1-H3-HA-gene-seg4-or-HA.txt`

```bash
zcat influenza.fna.gz | grep -P "^>.*A.*/(Ontario|Toronto|Canada-ON)/.*\(H(1|3)N\w\).*(segment 4|\(HA\)|hemagglutinin)" > ontario-H1-H3-HA-gene-seg4-or-HA.txt
```

Extract GI/NCBI ID from FASTA headers with regex

In [15]:
header = '>gi|52078172|gb|AY619969|Influenza A virus (A/swine/Ontario/K01477/01(H3N3)) hemagglutinin (HA) gene, complete cds'

Check that the number after `>gi|` can be parsed

In [16]:
REGEX_GI = re.compile(r'>gi\|(\d+)')
m = REGEX_GI.match(header)
if m:
    print(m.group(1))

52078172


Parse all GIs

In [17]:
gis = []
with open('ontario-H1-H3-HA-gene-seg4-or-HA.txt') as f:
    for l in f:
        m = REGEX_GI.match(l)
        if m:
            gis.append(m.group(1))

Should have 892 unique GIs

In [73]:
assert len(set(gis)) == len(gis)
len(gis)

892

Fetch Genbank entries for each of the 892 GI/NCBI IDs 

In [19]:
with Entrez.efetch(db='nucleotide',
                   id=gis,
                   rettype='gb',
                   retmode='text') as efetch_handle:
    entrez_gb_recs = [x for x in SeqIO.parse(efetch_handle, 'genbank')]

Should have retrieved 892 Genbank records

In [20]:
len(entrez_gb_recs)

892

Peek into one of the Genbank entries

In [21]:
gb_rec = entrez_gb_recs[0]

In [23]:
gb_rec

SeqRecord(seq=Seq('ATGAAGACCATTATTGTTCTGAGTTGTTTTTTCTGTCTGGCTTTCAGCCAAAAT...TGA', IUPACAmbiguousDNA()), id='AY619969.1', name='AY619969', description='Influenza A virus (A/swine/Ontario/K01477/01(H3N3)) hemagglutinin (HA) gene, complete cds', dbxrefs=[])

In [25]:
gb_rec.name

'AY619969'

In [24]:
gb_rec.description

'Influenza A virus (A/swine/Ontario/K01477/01(H3N3)) hemagglutinin (HA) gene, complete cds'

In [22]:
gb_rec.annotations

{'molecule_type': 'RNA',
 'topology': 'linear',
 'data_file_division': 'VRL',
 'date': '28-DEC-2004',
 'accessions': ['AY619969'],
 'sequence_version': 1,
 'keywords': [''],
 'source': 'Influenza A virus (A/swine/Ontario/K01477/01(H3N3))',
 'organism': 'Influenza A virus (A/swine/Ontario/K01477/01(H3N3))',
 'taxonomy': ['Viruses',
  'ssRNA viruses',
  'ssRNA negative-strand viruses',
  'Orthomyxoviridae',
  'Influenzavirus A'],
 'references': [Reference(title='Characterization of avian H3N3 and H1N1 influenza A viruses isolated from pigs in Canada', ...),
  Reference(title='Direct Submission', ...)]}

In [27]:
gb_rec.features

[SeqFeature(FeatureLocation(ExactPosition(0), ExactPosition(1701), strand=1), type='source'),
 SeqFeature(FeatureLocation(ExactPosition(0), ExactPosition(1701), strand=1), type='gene'),
 SeqFeature(FeatureLocation(ExactPosition(0), ExactPosition(1701), strand=1), type='CDS')]

In [74]:
gb_rec.features[0].qualifiers

OrderedDict([('organism',
              ['Influenza A virus (A/swine/Ontario/K01477/01(H3N3))']),
             ('mol_type', ['genomic RNA']),
             ('strain', ['A/swine/Ontario/K01477/01']),
             ('serotype', ['H3N3']),
             ('db_xref', ['taxon:292589'])])

Check that all Genbank records have a source feature as the first sequence feature

In [28]:
all([x.features[0].type == 'source' for x in entrez_gb_recs])

True

Parse metadata from Genbank source sequence feature qualifiers

In [29]:
def parse_gb_md(rec):
    source_feature = rec.features[0]
    assert source_feature.type == 'source', rec
    return {k:v[0] for k,v in rec.features[0].qualifiers.items()}

In [32]:
def genbank_md(rec): 
    out = parse_gb_md(rec)
    out['accession'] = rec.id
    return out

Convert list of metadata dicts to Pandas DataFrame

In [33]:
df_gb_md_892 = pd.DataFrame([genbank_md(x) for x in entrez_gb_recs])

Parse/coerce `collection_date` values into standard DateTime format (`pd.Timestamp`)

In [43]:
dates = pd.to_datetime(df_gb_md_892.collection_date, errors='coerce')
years = [x.year if not pd.isnull(x) else None for x in dates]
df_gb_md_892['collection_year'] = years
df_gb_md_892.collection_date = [str(x).split()[0] if not pd.isnull(x) else None for x in dates]

Set values for collection year, month and day in DataFrame

In [45]:
df_gb_md_892['collection_month'] = dates.dt.month
df_gb_md_892['collection_day'] = dates.dt.day

In [51]:
dts = pd.to_datetime(df_gb_md_892.collection_date)

### Compute flu season

Come up with a function to output flu season value where the collection date is used to derive the flu season based on the following:

- if month is August (8) or less, then flu season is `{year - 1}-{year}`
- otherwise, flu season is `{year}-{year + 1}`

For example,
- a collection date of 2018-05-01 should have a flu season of 2017-2018
- a collection date of 2016-10-01 should have a flu season of 2016-2017

In [53]:
dt = dts.loc[891]

In [54]:
dt

Timestamp('2018-05-01 00:00:00')

In [57]:
dt.month

5

In [58]:
def flu_season(dt):
    m = dt.month
    y = dt.year
    return f'{y}-{y+1}' if m > 8 else f'{y-1}-{y}'

In [59]:
flu_season(dt)

'2017-2018'

Add flu season for each non-null collection date in the dataframe

In [60]:
flu_seasons = [flu_season(dt) if not pd.isnull(dt) else None for dt in dts]

In [62]:
df_gb_md_892['flu_season'] = flu_seasons

Sort by collection date in descending order (most recent to least)

In [64]:
df_gb_md_892.sort_values('collection_date', inplace=True, ascending=False)

Peek at DataFrame of sequence metadata

In [65]:
df_gb_md_892

,organism,mol_type,strain,serotype,db_xref,accession,country,host,segment,collection_date,isolation_source,isolate,note,lab_host,lat_lon,collection_year,collection_month,collection_day,flu_season
891,Influenza A virus,viral cRNA,A/swine/Ontario/SD0298/2018,H3N2,taxon:11320,MK462790.1,Canada: Ontario,swine,4,2018-05-01,MDCK cells,NaN,NaN,NaN,NaN,2018.0,5.0,1.0,2017-2018
744,Influenza A virus,viral cRNA,A/Ontario/026/2018,H3N2,taxon:11320,MG889769.1,Canada: Ontario,Homo sapiens,4,2018-01-11,nasopharyngeal swab,026,original specimen,NaN,NaN,2018.0,1.0,11.0,2017-2018
752,Influenza A virus,viral cRNA,A/Ontario/034/2018,H3N2,taxon:11320,MG889777.1,Canada: Ontario,Homo sapiens,4,2018-01-11,nasopharyngeal swab,034,original specimen,NaN,NaN,2018.0,1.0,11.0,2017-2018
751,Influenza A virus,viral cRNA,A/Ontario/033/2018,H3N2,taxon:11320,MG889776.1,Canada: Ontario,Homo sapiens,4,2018-01-11,nasopharyngeal swab,033,original specimen,NaN,NaN,2018.0,1.0,11.0,2017-2018
754,Influenza A virus,viral cRNA,A/Ontario/038/2018,H3N2,taxon:11320,MG889779.1,Canada: Ontario,Homo sapiens,4,2018-01-10,nasopharyngeal swab,038,original specimen,NaN,NaN,2018.0,1.0,10.0,2017-2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11,Influenza A virus (A/Ontario/RV1273/2005(H3N2)),genomic DNA,A/Ontario/RV1273/2005,H3N2,taxon:381529,DQ469962.1,Canada,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
12,Influenza A virus (A/swine/Ontario/33853/2005(...,genomic DNA,A/swine/Ontario/33853/2005,H3N2,taxon:381533,DQ469994.1,Canada,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
13,Influenza A virus (A/turkey/Ontario/31232/2005...,genomic DNA,A/turkey/Ontario/31232/2005,H3N2,taxon:381534,DQ470002.1,Canada,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
14,Influenza A virus (A/swine/Ontario/00130/97(H3...,genomic RNA,A/swine/Ontario/00130/97,NaN,taxon:133777,AF251395.2,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None


### Save entire dataframe to CSV file

In [66]:
df_gb_md_892.to_csv('2019-09-26-McLaughlin-NCBI-Influenza-DB-H3-or-H1-Ontario-metadata-from-genbank.csv', index=False)

### Filter for human-derived samples

Frequencies of distinct host values

In [67]:
df_gb_md_892.host.value_counts()

Homo sapiens                       698
swine                               27
turkey                              10
human; gender M; age 25              6
Swine                                6
                                  ... 
Homo sapiens; gender M; age 84Y      1
Homo sapiens; gender M; age 9Y       1
human; gender M; age 0               1
Homo sapiens; gender F; age 28Y      1
human; gender M; age 55              1
Name: host, Length: 109, dtype: int64

IAV with host info not provided in GenBank file source feature

In [69]:
df_gb_md_892[pd.isnull(df_gb_md_892.host)]

,organism,mol_type,strain,serotype,db_xref,accession,country,host,segment,collection_date,isolation_source,isolate,note,lab_host,lat_lon,collection_year,collection_month,collection_day,flu_season
0,Influenza A virus (A/swine/Ontario/K01477/01(H...,genomic RNA,A/swine/Ontario/K01477/01,H3N3,taxon:292589,AY619969.1,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
1,Influenza A virus (A/swine/Ontario/42729A/01(H...,genomic RNA,A/swine/Ontario/42729A/01,H3N3,taxon:292590,AY619977.1,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
2,Influenza A virus (A/swine/Ontario/Biovet1/05(...,genomic RNA,A/swine/Ontario/Biovet1/05,H3N2,taxon:354557,DQ241762.1,Canada,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
3,Influenza A virus (A/swine/Ontario/57561/03(H1...,genomic RNA,A/swine/Ontario/57561/03,H1N1,taxon:358575,DQ280195.1,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
4,Influenza A virus (A/swine/Ontario/55383/04(H1...,genomic RNA,A/swine/Ontario/55383/04,H1N2,taxon:358580,DQ280212.1,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
5,Influenza A virus (A/swine/Ontario/53518/03(H1...,genomic RNA,A/swine/Ontario/53518/03,H1N1,taxon:358577,DQ280219.1,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
6,Influenza A virus (A/swine/Ontario/52156/03(H1...,genomic RNA,A/swine/Ontario/52156/03,H1N2,taxon:358581,DQ280227.1,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
7,Influenza A virus (A/swine/Ontario/48235/04(H1...,genomic RNA,A/swine/Ontario/48235/04,H1N2,taxon:358582,DQ280236.1,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
8,Influenza A virus (A/swine/Ontario/23866/04(H1...,genomic RNA,A/swine/Ontario/23866/04,H1N1,taxon:358578,DQ280243.1,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
9,Influenza A virus (A/swine/Ontario/11112/2004(...,genomic RNA,A/swine/Ontario/11112/04,H1N1,taxon:358579,DQ280250.1,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None


In [76]:
human_regex_pattern = r'.*([Hh]uman|[Hh]omo).*'

In [77]:
df_gb_md_892.host.str.match(human_regex_pattern).sum()

828

What does the metadata for the non-human regex pattern matching sequences look like?

In [85]:
for i,r in df_gb_md_892[~df_gb_md_892.host.str.match(human_regex_pattern, na=False)].iterrows():
    print(f'{r.strain: <40} {r.host: <20} {r.collection_date}')

A/swine/Ontario/SD0298/2018              swine                2018-05-01
A/swine/Ontario/DM_21/2017               swine                2017-04-26
A/swine/Ontario/DM_11/2017               swine                2017-01-16
A/turkey/Ontario/FAV-006-4/2016          turkey               2016-04-07
A/turkey/Ontario/FAV-006-10/2016         turkey               2016-04-07
A/turkey/Ontario/FAV-005-2/2016          turkey               2016-04-05
A/swine/Ontario/G3/2014                  swine                2014-11-07
A/swine/Ontario/G10/2014                 swine                2014-09-04
A/swine/Ontario/G12/2014                 swine                2014-09-03
A/swine/Ontario/G13/2014                 swine                2014-09-03
A/swine/Ontario/G14/2014                 swine                2014-08-12
A/swine/Ontario/G16/2014                 swine                2014-07-24
A/swine/Ontario/G11/2014                 swine                2014-07-15
A/swine/Ontario/G15/2014                 swine     

In [86]:
df_human = df_gb_md_892[df_gb_md_892.host.str.match(human_regex_pattern, na=False)]

In [93]:
df_human.host.value_counts().to_dict()

{'Homo sapiens': 698,
 'human; gender M; age 25': 6,
 'Homo sapiens; gender F; age 22': 4,
 'Homo sapiens; gender M; age 21Y': 3,
 'Homo sapiens; gender M; age 26Y': 3,
 'human; gender F; age 39': 2,
 'human; gender F; age 25': 2,
 'Homo sapiens; gender M; age 83Y': 2,
 'human; gender F; age 14': 2,
 'human; gender M; age 17': 2,
 'Homo sapiens; gender M; age 79Y': 2,
 'Homo sapiens; gender M; age 95Y': 2,
 'human; gender F; age 33': 2,
 'Homo sapiens; gender M; age 86Y': 2,
 'human; gender F; age 5': 2,
 'Homo sapiens; gender M; age 73Y': 2,
 'Homo sapiens; gender M; age 76Y': 2,
 'human; gender F; age 24': 2,
 'Homo sapiens; gender F; age 91Y': 2,
 'Homo sapiens; gender M; age 2Y': 2,
 'Homo sapiens; gender M; age 4Y': 2,
 'Homo sapiens; gender M; age 19Y': 1,
 'human; gender M; age 5': 1,
 'Homo sapiens; gender M; age 32Y': 1,
 'Homo sapiens; gender M; age 11mo': 1,
 'human; gender M; age 14': 1,
 'human; gender F; age 88': 1,
 'Homo sapiens; gender F; age 21Y': 1,
 'human; gender M

In [94]:
df_human.to_csv('2019-09-26-McLaughlin-NCBI-Influenza-DB-H3-or-H1-Ontario-Human-metadata-from-genbank.csv', index=False)

## Write all Genbank records to a file

In [36]:
SeqIO.write(entrez_gb_recs, '2019-09-26-McLaughlin-NCBI-Influenza-DB-H3-or-H1-Ontario-metadata-from-genbank.gb', 'genbank')

892

## Partition human sequences by flu season

In [103]:
id_to_gb = {x.id:x for x in entrez_gb_recs}

In [95]:
df_human.flu_season.value_counts()

2015-2016    210
2016-2017    163
2014-2015    135
2010-2011     67
2012-2013     57
2009-2010     54
2017-2018     50
2008-2009     40
2011-2012     30
2013-2014     16
2007-2008      6
Name: flu_season, dtype: int64

Group by `flu_season` and `serotype`

In [110]:
g = df_human.groupby(['flu_season','serotype'])

Create lists of accessions grouped by `flu_season` and `serotype`

In [111]:
grouped_acc = g.accession.apply(list)

In [112]:
grouped_acc

flu_season  serotype
2007-2008   H1N1        [FJ800811.1, FJ800819.1, FJ800810.1, FJ800818....
2008-2009   H1N1        [HQ239567.1, CY060534.2, CY060502.2, CY060526....
2009-2010   H1N1        [CY081093.2, HQ239460.2, CY060726.2, HQ239459....
            H3N2                     [JQ658889.1, JQ658888.1, CY054550.1]
2010-2011   H1N1        [CY081101.2, CY081069.2, CY081085.2, CY081077....
            H3N2        [CY111003.1, CY111002.1, CY111001.1, CY111000....
2011-2012   H1N1        [JX875001.1, KF551079.1, KF551095.1, KF551093....
            H3N2        [KF551077.1, KF551078.1, KF551076.1, KF551075....
2012-2013   H1N1        [KF886379.1, KF886366.1, KF886376.1, KF886371....
            H3N2        [KJ734749.1, KF886354.1, KJ734748.1, KJ734747....
2013-2014   H1N1        [KP864399.1, KP864397.1, KP864398.1, KP864396....
            H3N2                     [KP864423.1, KP864424.1, KP864425.1]
2014-2015   H3N2        [KU729355.1, KU729354.1, KU729353.1, KU729458....
2015-2016   H1N1 

### Write partitioned human HA sequences to GenBank and FASTA files

In [109]:
!mkdir human-HA-sequences-by-flu-season

In [117]:
for (flu_season, serotype), accessions in grouped_acc.items():
    print(f'{serotype}| Flu season: {flu_season} (N={len(accessions)})')
    gbs = [id_to_gb[acc] for acc in accessions]
    filename = f'human-HA-sequences-by-flu-season/HA-{serotype}-flu_season-{flu_season}'
    SeqIO.write(gbs, f'{filename}.gb', 'genbank')
    SeqIO.write(gbs, f'{filename}.fa', 'fasta')

H1N1| Flu season: 2007-2008 (N=6)
H1N1| Flu season: 2008-2009 (N=40)
H1N1| Flu season: 2009-2010 (N=51)
H3N2| Flu season: 2009-2010 (N=3)
H1N1| Flu season: 2010-2011 (N=5)
H3N2| Flu season: 2010-2011 (N=62)
H1N1| Flu season: 2011-2012 (N=18)
H3N2| Flu season: 2011-2012 (N=12)
H1N1| Flu season: 2012-2013 (N=18)
H3N2| Flu season: 2012-2013 (N=39)
H1N1| Flu season: 2013-2014 (N=13)
H3N2| Flu season: 2013-2014 (N=3)
H3N2| Flu season: 2014-2015 (N=135)
H1N1| Flu season: 2015-2016 (N=198)
H3N2| Flu season: 2015-2016 (N=12)
H1N1| Flu season: 2016-2017 (N=1)
H3N2| Flu season: 2016-2017 (N=162)
H3N2| Flu season: 2017-2018 (N=50)


Peek into output directory at `human-HA-sequences-by-flu-season/`

In [118]:
!ls -lh human-HA-sequences-by-flu-season/

total 5.4M
-rw-r--r-- 1 pkruczkiewicz grp_pkruczkiewicz 5.7K Sep 26 11:43 HA-H1N1-flu_season-2007-2008.fa
-rw-r--r-- 1 pkruczkiewicz grp_pkruczkiewicz  21K Sep 26 11:43 HA-H1N1-flu_season-2007-2008.gb
-rw-r--r-- 1 pkruczkiewicz grp_pkruczkiewicz  67K Sep 26 11:43 HA-H1N1-flu_season-2008-2009.fa
-rw-r--r-- 1 pkruczkiewicz grp_pkruczkiewicz 206K Sep 26 11:43 HA-H1N1-flu_season-2008-2009.gb
-rw-r--r-- 1 pkruczkiewicz grp_pkruczkiewicz  91K Sep 26 11:43 HA-H1N1-flu_season-2009-2010.fa
-rw-r--r-- 1 pkruczkiewicz grp_pkruczkiewicz 279K Sep 26 11:43 HA-H1N1-flu_season-2009-2010.gb
-rw-r--r-- 1 pkruczkiewicz grp_pkruczkiewicz 9.2K Sep 26 11:43 HA-H1N1-flu_season-2010-2011.fa
-rw-r--r-- 1 pkruczkiewicz grp_pkruczkiewicz  27K Sep 26 11:43 HA-H1N1-flu_season-2010-2011.gb
-rw-r--r-- 1 pkruczkiewicz grp_pkruczkiewicz  33K Sep 26 11:43 HA-H1N1-flu_season-2011-2012.fa
-rw-r--r-- 1 pkruczkiewicz grp_pkruczkiewicz 103K Sep 26 11:43 HA-H1N1-flu_season-2011-2012.gb
-rw-r--r-- 1 pkruczkiewicz grp_pkruczki

### MAFFT multiple sequence alignment (MSA) of partitioned human HA genes

Perform MSA with MAFFT (v7.407) of each set of sequences with the L-INS-i strategy for high accuracy. 

In [119]:
!mafft --version

v7.407 (2018/Jul/23)


In [121]:
!mafft --help

------------------------------------------------------------------------------
  MAFFT v7.407 (2018/Jul/23)
  https://mafft.cbrc.jp/alignment/software/
  MBE 30:772-780 (2013), NAR 30:3059-3066 (2002)
------------------------------------------------------------------------------
High speed:
  % mafft in > out
  % mafft --retree 1 in > out (fast)

High accuracy (for <~200 sequences x <~2,000 aa/nt):
  % mafft --maxiterate 1000 --localpair  in > out (% linsi in > out is also ok)
  % mafft --maxiterate 1000 --genafpair  in > out (% einsi in > out)
  % mafft --maxiterate 1000 --globalpair in > out (% ginsi in > out)

If unsure which option to use:
  % mafft --auto in > out

--op # :         Gap opening penalty, default: 1.53
--ep # :         Offset (works like gap extension penalty), default: 0.0
--maxiterate # : Maximum number of iterative refinement, default: 0
--clustalout :   Output: clustal format, default: fasta
--reorder :      Outorder: aligned, default: input order
--quiet :      

In [120]:
!mkdir human-HA-sequences-by-flu-season/mafft-msa

Use GNU Parallel to run parallel instances of `mafft` MSA

In [123]:
!parallel -v mafft --thread -1 --maxiterate 1000 --localpair {} ">" human-HA-sequences-by-flu-season/mafft-msa/{/} ::: human-HA-sequences-by-flu-season/*.fa

mafft --thread -1 --maxiterate 1000 --localpair human-HA-sequences-by-flu-season/HA-H1N1-flu_season-2016-2017.fa > human-HA-sequences-by-flu-season/mafft-msa/HA-H1N1-flu_season-2016-2017.fa
OS = linux
The number of physical cores =  28
outputhat23=16
treein = 0
compacttree = 0
minimumweight = 0.000010
autosubalignment = 0.000000
nthread = 8
randomseed = 0
blosum 62 / kimura 200
poffset = 0
niter = 16
sueff_global = 0.100000
nadd = 16

Strategy:
 L-INS-i (Probably most accurate, very slow)
 Iterative refinement method (<16) with LOCAL pairwise alignment information

If unsure which option to use, try 'mafft --auto input > output'.
For more information, see 'mafft --help', 'mafft --man' and the mafft page.

The default gap scoring scheme has been changed in version 7.110 (2013 Oct).
It tends to insert more gaps into gap-rich regions than previous versions.
To disable this change, add the --leavegappyregion option.

mafft --thread -1 --maxiterate 1000 --localpair human-HA-sequences-by-flu

mafft --thread -1 --maxiterate 1000 --localpair human-HA-sequences-by-flu-season/HA-H1N1-flu_season-2013-2014.fa > human-HA-sequences-by-flu-season/mafft-msa/HA-H1N1-flu_season-2013-2014.fa
OS = linux
The number of physical cores =  28
outputhat23=16
treein = 0
compacttree = 0
stacksize: 8192 kb
generating a scoring matrix for nucleotide (dist=200) ... done
All-to-all alignment.
tbfast-pair (nuc) Version 7.407
alg=L, model=DNA200 (2), 2.00 (6.00), -0.10 (-0.30), noshift, amax=0.0
28 thread(s)

outputhat23=16
Loading 'hat3.seed' ... 
done.
Writing hat3 for iterative refinement
generating a scoring matrix for nucleotide (dist=200) ... done
Gap Penalty = -1.53, +0.00, +0.00
tbutree = 1, compacttree = 0
Constructing a UPGMA tree ... 
   10 / 13
done.

Progressive alignment ... 
STEP    12 /12 (thread   11) 
done.
tbfast (nuc) Version 7.407
alg=A, model=DNA200 (2), 1.53 (4.59), -0.00 (-0.00), noshift, amax=0.0
16 thread(s)

minimumweight = 0.000010
autosubalignment = 0.000000
nthread = 8
ra

mafft --thread -1 --maxiterate 1000 --localpair human-HA-sequences-by-flu-season/HA-H3N2-flu_season-2012-2013.fa > human-HA-sequences-by-flu-season/mafft-msa/HA-H3N2-flu_season-2012-2013.fa
OS = linux
The number of physical cores =  28
outputhat23=16
treein = 0
compacttree = 0
stacksize: 8192 kb
generating a scoring matrix for nucleotide (dist=200) ... done
All-to-all alignment.
tbfast-pair (nuc) Version 7.407
alg=L, model=DNA200 (2), 2.00 (6.00), -0.10 (-0.30), noshift, amax=0.0
28 thread(s)

outputhat23=16
Loading 'hat3.seed' ... 
done.
Writing hat3 for iterative refinement
generating a scoring matrix for nucleotide (dist=200) ... done
Gap Penalty = -1.53, +0.00, +0.00
tbutree = 1, compacttree = 0
Constructing a UPGMA tree ... 
   30 / 39
done.

Progressive alignment ... 
STEP    38 /38 (thread    9) 
done.
tbfast (nuc) Version 7.407
alg=A, model=DNA200 (2), 1.53 (4.59), -0.00 (-0.00), noshift, amax=0.0
16 thread(s)

minimumweight = 0.000010
autosubalignment = 0.000000
nthread = 8
ra

mafft --thread -1 --maxiterate 1000 --localpair human-HA-sequences-by-flu-season/HA-H1N1-flu_season-2015-2016.fa > human-HA-sequences-by-flu-season/mafft-msa/HA-H1N1-flu_season-2015-2016.fa
OS = linux
The number of physical cores =  28
outputhat23=16
treein = 0
compacttree = 0
stacksize: 8192 kb
generating a scoring matrix for nucleotide (dist=200) ... done
All-to-all alignment.
tbfast-pair (nuc) Version 7.407
alg=L, model=DNA200 (2), 2.00 (6.00), -0.10 (-0.30), noshift, amax=0.0
28 thread(s)

outputhat23=16
Loading 'hat3.seed' ... 
done.
Writing hat3 for iterative refinement
generating a scoring matrix for nucleotide (dist=200) ... done
Gap Penalty = -1.53, +0.00, +0.00
tbutree = 1, compacttree = 0
Constructing a UPGMA tree ... 
  190 / 198
done.

Progressive alignment ... 
STEP   132 /197 (thread    7) 
Reallocating (by thread 15) ..done. *alloclen = 3096
STEP   197 /197 (thread    7) 
done.
tbfast (nuc) Version 7.407
alg=A, model=DNA200 (2), 1.53 (4.59), -0.00 (-0.00), noshift, amax

mafft --thread -1 --maxiterate 1000 --localpair human-HA-sequences-by-flu-season/HA-H3N2-flu_season-2014-2015.fa > human-HA-sequences-by-flu-season/mafft-msa/HA-H3N2-flu_season-2014-2015.fa
OS = linux
The number of physical cores =  28
outputhat23=16
treein = 0
compacttree = 0
stacksize: 8192 kb
generating a scoring matrix for nucleotide (dist=200) ... done
All-to-all alignment.
tbfast-pair (nuc) Version 7.407
alg=L, model=DNA200 (2), 2.00 (6.00), -0.10 (-0.30), noshift, amax=0.0
28 thread(s)

outputhat23=16
Loading 'hat3.seed' ... 
done.
Writing hat3 for iterative refinement
generating a scoring matrix for nucleotide (dist=200) ... done
Gap Penalty = -1.53, +0.00, +0.00
tbutree = 1, compacttree = 0
Constructing a UPGMA tree ... 
  130 / 135
done.

Progressive alignment ... 
STEP   134 /134 (thread    7) 
done.
tbfast (nuc) Version 7.407
alg=A, model=DNA200 (2), 1.53 (4.59), -0.00 (-0.00), noshift, amax=0.0
16 thread(s)

minimumweight = 0.000010
autosubalignment = 0.000000
nthread = 8


mafft --thread -1 --maxiterate 1000 --localpair human-HA-sequences-by-flu-season/HA-H3N2-flu_season-2016-2017.fa > human-HA-sequences-by-flu-season/mafft-msa/HA-H3N2-flu_season-2016-2017.fa
OS = linux
The number of physical cores =  28
outputhat23=16
treein = 0
compacttree = 0
stacksize: 8192 kb
generating a scoring matrix for nucleotide (dist=200) ... done
All-to-all alignment.
tbfast-pair (nuc) Version 7.407
alg=L, model=DNA200 (2), 2.00 (6.00), -0.10 (-0.30), noshift, amax=0.0
28 thread(s)

outputhat23=16
Loading 'hat3.seed' ... 
done.
Writing hat3 for iterative refinement
generating a scoring matrix for nucleotide (dist=200) ... done
Gap Penalty = -1.53, +0.00, +0.00
tbutree = 1, compacttree = 0
Constructing a UPGMA tree ... 
  160 / 162
done.

Progressive alignment ... 
STEP   161 /161 (thread   14) 
done.
tbfast (nuc) Version 7.407
alg=A, model=DNA200 (2), 1.53 (4.59), -0.00 (-0.00), noshift, amax=0.0
16 thread(s)

minimumweight = 0.000010
autosubalignment = 0.000000
nthread = 8


In [124]:
!mkdir human-HA-sequences-by-flu-season/concat

In [125]:
!cat human-HA-sequences-by-flu-season/HA-H1N1*.fa > human-HA-sequences-by-flu-season/concat/HA-H1N1-all-seasons.fa
!cat human-HA-sequences-by-flu-season/HA-H3N2*.fa > human-HA-sequences-by-flu-season/concat/HA-H3N2-all-seasons.fa

In [126]:
!mkdir human-HA-sequences-by-flu-season/concat/mafft-msa

In [127]:
!parallel -v mafft --thread -1 --maxiterate 1000 --localpair {} ">" human-HA-sequences-by-flu-season/concat/mafft-msa/{/} ::: human-HA-sequences-by-flu-season/concat/*.fa

mafft --thread -1 --maxiterate 1000 --localpair human-HA-sequences-by-flu-season/concat/HA-H1N1-all-seasons.fa > human-HA-sequences-by-flu-season/concat/mafft-msa/HA-H1N1-all-seasons.fa
OS = linux
The number of physical cores =  28
outputhat23=16
treein = 0
compacttree = 0
stacksize: 8192 kb
generating a scoring matrix for nucleotide (dist=200) ... done
All-to-all alignment.
tbfast-pair (nuc) Version 7.407
alg=L, model=DNA200 (2), 2.00 (6.00), -0.10 (-0.30), noshift, amax=0.0
28 thread(s)

outputhat23=16
Loading 'hat3.seed' ... 
done.
Writing hat3 for iterative refinement
generating a scoring matrix for nucleotide (dist=200) ... done
Gap Penalty = -1.53, +0.00, +0.00
tbutree = 1, compacttree = 0
Constructing a UPGMA tree ... 
  340 / 350
done.

Progressive alignment ... 
STEP   342 /349 (thread    8) 
Reallocating (by thread 6) ..done. *alloclen = 4548
STEP   349 /349 (thread   14) 
done.
tbfast (nuc) Version 7.407
alg=A, model=DNA200 (2), 1.53 (4.59), -0.00 (-0.00), noshift, amax=0.0


mafft --thread -1 --maxiterate 1000 --localpair human-HA-sequences-by-flu-season/concat/HA-H3N2-all-seasons.fa > human-HA-sequences-by-flu-season/concat/mafft-msa/HA-H3N2-all-seasons.fa
OS = linux
The number of physical cores =  28
outputhat23=16
treein = 0
compacttree = 0
stacksize: 8192 kb
generating a scoring matrix for nucleotide (dist=200) ... done
All-to-all alignment.
tbfast-pair (nuc) Version 7.407
alg=L, model=DNA200 (2), 2.00 (6.00), -0.10 (-0.30), noshift, amax=0.0
28 thread(s)

outputhat23=16
Loading 'hat3.seed' ... 
done.
Writing hat3 for iterative refinement
generating a scoring matrix for nucleotide (dist=200) ... done
Gap Penalty = -1.53, +0.00, +0.00
tbutree = 1, compacttree = 0
Constructing a UPGMA tree ... 
  470 / 478
done.

Progressive alignment ... 
STEP   477 /477 (thread    7) 
done.
tbfast (nuc) Version 7.407
alg=A, model=DNA200 (2), 1.53 (4.59), -0.00 (-0.00), noshift, amax=0.0
16 thread(s)

minimumweight = 0.000010
autosubalignment = 0.000000
nthread = 8
rand